# Guardian API

In [ ]:
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_data_dir = '/content/drive/MyDrive/Colab Notebooks/vkr/'

Mounted at /content/drive


In [ ]:
idioms = pd.read_csv(os.path.join(drive_data_dir, 'unique_idioms_filtered.csv'))  # подгрузим те идиомы, которые у нас остались после фильтрации
idioms = idioms.rename(columns={'0': 'idiom'})

In [ ]:
idioms

,idiom
0,never mind
1,next to nothing
2,nine times out of ten
3,no spring chicken
4,not in a million years
...,...
168,round the clock
169,rule of thumb
170,you bet
171,silver bullet


In [ ]:
idioms['query'] = idioms['idiom'].apply(lambda x: '%22' + '%20'.join(x.split()) + '%22')  # создадим столбец query (нужен для создания запросов для api)

In [ ]:
idioms.head(3)  

,idiom,query
0,never mind,%22never%20mind%22
1,next to nothing,%22next%20to%20nothing%22
2,nine times out of ten,%22nine%20times%20out%20of%20ten%22


In [ ]:
!git clone https://github.com/prabhath6/theguardian-api-python.git
%cd theguardian-api-python

# !source /content/theguardian-api-python/guardian/bin/activate
!pip install -r requirements.txt -q

Cloning into 'theguardian-api-python'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 36.88 KiB | 609.00 KiB/s, done.
Resolving deltas: 100% (70/70), done.
/content/theguardian-api-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.18 requires requests>=2.26, but you have requests 2.22.0 which is incompatible.
tweepy 4.13.0 requires requests<3,>=2.27.0, but you have requests 2.22.0 which is incompatible.
google-colab 1.0.0 requires req

In [ ]:
from theguardian import theguardian_content

In [ ]:
%cd ..

/content


In [ ]:
api_key = 'a93689cf-7efb-4cb1-bf3b-e1bdfb4f75d0'

In [ ]:
# headers = {
#     'q': 'best of both worlds',
#     "page-size": 300,
#     "order-by": "relevance",
# }

# content = theguardian_content.Content(api=api_key, **headers)

In [ ]:
# create content
content = theguardian_content.Content(url='https://content.guardianapis.com/search?order-by=relevance&show-fields=bodyText&page-size=100&q=%22best%20of%20both%20worlds%22&', api=api_key)

# gets raw_response
raw_content = content.get_request_response()
print("Request Response status code {status}." .format(status=raw_content.status_code))
print("Request Response headers {header}." .format(header=raw_content.headers))

# # content
# print("Content Response headers {}." .format(content.response_headers()))

# # get all results of a page
# json_content = content.get_content_response()
# all_results = content.get_results(json_content)
# print("All results {}." .format(all_results))

# # actual response
# print("Response {response}" .format(response=json_content))

Request Response status code 200.
Request Response headers {'Date': 'Sun, 19 Mar 2023 17:54:40 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=GhOCqJeS1ClUkLebIjpeRhsfsJoVeBBekLlRHnOKD3QCJYWQ7aCTPLeLMk+FSUPtrd40XkdTasLbv1qIWVdhp/nqNJvBcQuLCc+jlpimansIVhZJDGQh0cToaxtf; Expires=Sun, 26 Mar 2023 17:54:40 GMT; Path=/, AWSALBCORS=GhOCqJeS1ClUkLebIjpeRhsfsJoVeBBekLlRHnOKD3QCJYWQ7aCTPLeLMk+FSUPtrd40XkdTasLbv1qIWVdhp/nqNJvBcQuLCc+jlpimansIVhZJDGQh0cToaxtf; Expires=Sun, 26 Mar 2023 17:54:40 GMT; Path=/; SameSite=None; Secure', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Cache-Control': 'max-age=0', 'Content-Encoding': 'gzip', 'Server': 'Concierge', 'X-RateLimit-Limit-day': '500', 'X-RateLimit-Remaining-minute': '59', 'X-RateLimit-Limit-minute': '60', 'X-RateLimit-Remaining-day': '499', 'X-Kong-Upstream-Latency': '295', 'X-Kong-Proxy-Latency': '0', 'Via': 'kong/1.5.1'}.


In [ ]:
[d['webTitle'] for d in content.get_results(content.get_content_response())]

In [ ]:
import spacy
sentencizer = spacy.load('en_core_web_sm')

# text = 'My first birthday was great. My 2. was even better.'
# [i.text.strip() for i in sentencizer(text).sents]

## Забираем из статьи все предложения с идиомой вместе с контекстом (2 соседних предложения)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

df = pd.DataFrame(columns=['idiom_label', 'sentence'])  # пустой датафрейм, куда будем добавлять примеры, которые спарсим для идиом

In [ ]:
%%time
total_parsed = 0
for i in range(len(idioms)):
  idiom, q = idioms.loc[i]  #.idiom, idioms.loc[i].query
  content = theguardian_content.Content(url=f'https://content.guardianapis.com/search?order-by=relevance&show-fields=bodyText&page-size=200&q={q}&',
                                        api=api_key)

  raw_content = content.get_request_response()
  if raw_content.status_code != 200:  # если запрос вернулся с ошибкой
    print(i, raw_content.status_code, q)  # выведем, что для этой идиомы guardian не смог ничего выдать
    continue
  try:  # обработка возможных исключений
    content_lst = content.get_results(content.get_content_response())
    cnt = 0  # счетчик собранных примеров для каждой идиомы
    for res in content_lst:
        if idiom in res['webTitle']:  # если идиома нашлась в заголовке, возьмем текст заголовка
            sentence = res['webTitle']
            df = df.append({'idiom_label': idiom, 'sentence': sentence}, ignore_index=True)
            total_parsed += 1
            cnt+= 1
        if idiom in res['fields']['bodyText']:
            article_text = res['fields']['bodyText']  # получим текст статьи
            sentences = [k.text.strip() for k in sentencizer(article_text).sents]
            idiom_sentences = []
            for j in range(len(sentences)):
                if idiom in sentences[j]:
                    idiom_sentences.append(sentences[j])
                    # if j > 0 and j < (len(sentences) - 1):
                    #     idiom_sentences.append(' '.join([sentences[j - 1], sentences[j], sentences[j + 1]]))
                    # elif j == 0:
                    #     idiom_sentences.append(' '.join([sentences[j], sentences[j + 1]]))
                    # elif j == (len(sentences) - 1):
                    #     idiom_sentences.append(' '.join([sentences[j - 1], sentences[j]]))
            for ex in idiom_sentences:
                df = df.append({'idiom_label': idiom, 'sentence': ex}, ignore_index=True)
                total_parsed += 1
                cnt += 1
  except Exception as e:
      print(e)
      continue
  print(i, idiom, ':', cnt)

print(f'Parsed {total_parsed} sentences in total.')

0 never mind : 89
1 next to nothing : 192
2 nine times out of ten : 51
3 no spring chicken : 111
4 not in a million years : 57
5 now or never : 178
6 dead and buried : 203
7 dead meat : 123
8 dead wood : 218
9 dirty work : 199
10 dressed up to the nines : 43
11 against time : 218
12 rags to riches : 54
13 rain or shine : 155
14 red ink : 196
15 bad blood : 156
16 bag of tricks : 192
17 baptism of fire : 197
18 behind bars : 234
19 behind closed doors : 235
20 bells and whistles : 180
21 big bucks : 169
22 big deal : 225
23 black and blue : 150
24 by the numbers : 1
25 face the music : 65
26 fall for it : 2
27 fall from grace : 196
28 food for thought : 140
29 for a song : 5
30 larger than life : 73
31 long in the tooth : 160
32 pain in the neck : 185
33 poetry in motion : 92
34 pressed for time : 129
35 safe and sound : 187
36 salt of the earth : 64
37 sink or swim : 144
38 skin and bones : 132
39 just what the doctor ordered : 138
40 in a nutshell : 172
41 in cold blood : 138
42 in th

In [ ]:
total_parsed / len(idioms)

144.35260115606937

В среднем получилось чуть больше 100 примеров на идиому

Посмотрим, не оказалось ли дубликатов

In [ ]:
print('Дублирующихся строчек датафрейма (идиома+предложение):', df.duplicated().sum())
print('Дублирующихся предложений:', df.sentence.duplicated().sum())

Дублирующихся строчек датафрейма (идиома+предложение): 293
Дублирующихся предложений: 313


In [ ]:
# df = df.drop(columns='idiom_label')
# df = df.rename(columns={'idiom': 'idiom_label'})
df = df[['idiom_label', 'sentence']]

In [ ]:
df[df.sentence.duplicated()]

,idiom_label,sentence
372,no spring chicken,He was no spring chicken but he worked extreme...
429,no spring chicken,"As a rough guideline, at 135C a 1.5kg chicken ..."
498,not in a million years,"'I'm never going to go back, not in a million ..."
531,now or never,IPCC report: ‘now or never’ if world is to sta...
532,now or never,The world can still hope to stave off the wors...
...,...,...
24772,silver bullet,They did put testing in but of course realised...
24805,silver bullet,"The Labour MP Margaret Hodge, who has long cam..."
24869,snowed under,(We will try to acknowledge them – but given h...
24871,snowed under,(We will try to acknowledge them – but given h...


Выкинем дублирующиеся предложения

In [ ]:
df1 = df.drop(index=df[df.sentence.duplicated()].index)
df1.shape[0]

24660

In [ ]:
df1.to_csv(os.path.join(drive_data_dir, 'guardian_parsed.csv'), index=False)